In [ ]:
import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packag

In [ ]:
import os
import pandas as pd
from google.colab import files

DEFAULT_PATH = "/mnt/data/final_dataset.csv"

def load_csv(default_path=DEFAULT_PATH):
    if os.path.exists(default_path):
        print("Loading dataset from default path...")
        return pd.read_csv(default_path)
    else:
        print("Upload the dataset:")
        uploaded = files.upload()
        fname = list(uploaded.keys())[0]
        return pd.read_csv(fname)

df = load_csv()
df.head()


Upload the dataset:


Saving final_dataset.csv to final_dataset.csv


,Date,Month,Year,Holidays_Count,Days,PM2.5,PM10,NO2,SO2,CO,Ozone,AQI
0,1,1,2021,0,5,408.80,442.42,160.61,12.95,2.77,43.19,462
1,2,1,2021,0,6,404.04,561.95,52.85,5.18,2.60,16.43,482
2,3,1,2021,1,7,225.07,239.04,170.95,10.93,1.40,44.29,263
3,4,1,2021,0,1,89.55,132.08,153.98,10.42,1.01,49.19,207
4,5,1,2021,0,2,54.06,55.54,122.66,9.70,0.64,48.88,149


In [ ]:
df["Datetime"] = pd.to_datetime(
    dict(year=df["Year"], month=df["Month"], day=df["Date"])
)
df = df.sort_values("Datetime").reset_index(drop=True)
df.head()


,Date,Month,Year,Holidays_Count,Days,PM2.5,PM10,NO2,SO2,CO,Ozone,AQI,Datetime
0,1,1,2021,0,5,408.80,442.42,160.61,12.95,2.77,43.19,462,2021-01-01
1,2,1,2021,0,6,404.04,561.95,52.85,5.18,2.60,16.43,482,2021-01-02
2,3,1,2021,1,7,225.07,239.04,170.95,10.93,1.40,44.29,263,2021-01-03
3,4,1,2021,0,1,89.55,132.08,153.98,10.42,1.01,49.19,207,2021-01-04
4,5,1,2021,0,2,54.06,55.54,122.66,9.70,0.64,48.88,149,2021-01-05


In [ ]:
def categorize_aqi(aqi):
    if aqi <= 50: return "Good"
    elif aqi <= 100: return "Satisfactory"
    elif aqi <= 200: return "Moderate"
    elif aqi <= 300: return "Poor"
    elif aqi <= 400: return "Very Poor"
    else: return "Severe"

df["AQI_Level"] = df["AQI"].apply(categorize_aqi)
df[["AQI", "AQI_Level"]].head()


,AQI,AQI_Level
0,462,Severe
1,482,Severe
2,263,Poor
3,207,Poor
4,149,Moderate


In [ ]:

def bin_pollutant(col, value):
    if value < df[col].quantile(0.33): return f"Low_{col}"
    elif value < df[col].quantile(0.66): return f"Medium_{col}"
    else: return f"High_{col}"

pollutants = ["PM2.5", "PM10", "NO2", "SO2", "CO", "Ozone"]

for col in pollutants:
    df[f"{col}_Level"] = df[col].apply(lambda x: bin_pollutant(col, x))

df[[
    "PM2.5_Level","PM10_Level","NO2_Level",
    "SO2_Level","CO_Level","Ozone_Level","AQI_Level"
]].head()


,PM2.5_Level,PM10_Level,NO2_Level,SO2_Level,CO_Level,Ozone_Level,AQI_Level
0,High_PM2.5,High_PM10,High_NO2,Medium_SO2,High_CO,High_Ozone,Severe
1,High_PM2.5,High_PM10,High_NO2,Low_SO2,High_CO,Low_Ozone,Severe
2,High_PM2.5,Medium_PM10,High_NO2,Medium_SO2,High_CO,High_Ozone,Poor
3,Medium_PM2.5,Low_PM10,High_NO2,Medium_SO2,Medium_CO,High_Ozone,Poor
4,Medium_PM2.5,Low_PM10,High_NO2,Medium_SO2,Low_CO,High_Ozone,Moderate


In [ ]:

from mlxtend.preprocessing import TransactionEncoder

transactions = []

for _, row in df.iterrows():
    items = [
        row["PM2.5_Level"],
        row["PM10_Level"],
        row["NO2_Level"],
        row["SO2_Level"],
        row["CO_Level"],
        row["Ozone_Level"],
        row["AQI_Level"]
    ]
    transactions.append(items)

te = TransactionEncoder()
te_array = te.fit(transactions).transform(transactions)

df_trans = pd.DataFrame(te_array, columns=te.columns_)
df_trans.head()


,Good,High_CO,High_NO2,High_Ozone,High_PM10,High_PM2.5,High_SO2,Low_CO,Low_NO2,Low_Ozone,...,Medium_NO2,Medium_Ozone,Medium_PM10,Medium_PM2.5,Medium_SO2,Moderate,Poor,Satisfactory,Severe,Very Poor
0,False,True,True,True,True,True,False,False,False,False,...,False,False,False,False,True,False,False,False,True,False
1,False,True,True,False,True,True,False,False,False,True,...,False,False,False,False,False,False,False,False,True,False
2,False,True,True,True,False,True,False,False,False,False,...,False,False,True,False,True,False,True,False,False,False
3,False,False,True,True,False,False,False,False,False,False,...,False,False,False,True,True,False,True,False,False,False
4,False,False,True,True,False,False,False,True,False,False,...,False,False,False,True,True,True,False,False,False,False


In [ ]:
from mlxtend.frequent_patterns import apriori

frequent_itemsets = apriori(
    df_trans,
    min_support=0.10,
    use_colnames=True
)

frequent_itemsets.sort_values("support", ascending=False).head(10)


,support,itemsets
0,0.344969,(High_CO)
1,0.340178,(High_NO2)
2,0.340178,(High_Ozone)
3,0.340178,(High_PM10)
4,0.340178,(High_PM2.5)
5,0.340178,(High_SO2)
12,0.333333,(Medium_CO)
16,0.330595,(Medium_PM2.5)
17,0.329911,(Medium_SO2)
9,0.329911,(Low_PM10)


In [ ]:
from mlxtend.frequent_patterns import association_rules

rules = association_rules(
    frequent_itemsets,
    metric="confidence",
    min_threshold=0.6
)

rules = rules.sort_values("confidence", ascending=False)
rules.head(10)


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski
54,"(Satisfactory, Low_CO)",(Low_PM10),0.120465,0.329911,0.119781,0.994318,3.013898,1.0,0.080038,117.935661,0.759724,0.362319,0.991521,0.678694
83,"(Satisfactory, Low_PM2.5, Low_CO)",(Low_PM10),0.116359,0.329911,0.115674,0.994118,3.013290,1.0,0.077286,113.915127,0.756118,0.349896,0.991222,0.672370
16,(Satisfactory),(Low_PM10),0.182752,0.329911,0.180698,0.988764,2.997063,1.0,0.120406,59.637919,0.815346,0.544330,0.983232,0.768241
64,"(Satisfactory, Low_PM2.5)",(Low_PM10),0.169062,0.329911,0.167009,0.987854,2.994305,1.0,0.111233,55.170659,0.801543,0.503093,0.981874,0.747039
59,"(Satisfactory, Low_CO)",(Low_PM2.5),0.120465,0.329227,0.116359,0.965909,2.933874,1.0,0.076698,19.676021,0.749435,0.349076,0.949177,0.659670
82,"(Satisfactory, Low_PM10, Low_CO)",(Low_PM2.5),0.119781,0.329227,0.115674,0.965714,2.933282,1.0,0.076239,19.564225,0.748774,0.347023,0.948886,0.658533
87,"(Satisfactory, Low_CO)","(Low_PM10, Low_PM2.5)",0.120465,0.271047,0.115674,0.960227,3.542657,1.0,0.083022,18.327955,0.816029,0.419355,0.945439,0.693497
29,"(High_CO, Very Poor)",(High_PM10),0.123888,0.340178,0.114990,0.928177,2.728504,1.0,0.072846,9.186753,0.723079,0.329412,0.891148,0.633102
17,(Satisfactory),(Low_PM2.5),0.182752,0.329227,0.169062,0.925094,2.809900,1.0,0.108896,8.954825,0.788151,0.493014,0.888328,0.719304
63,"(Satisfactory, Low_PM10)",(Low_PM2.5),0.180698,0.329227,0.167009,0.924242,2.807314,1.0,0.107518,8.854209,0.785776,0.487026,0.887059,0.715759


In [ ]:
aqi_rules = rules[rules["consequents"].astype(str).str.contains("AQI")]
aqi_rules = aqi_rules.sort_values("lift", ascending=False)

print("Top Rules Explaining AQI Behavior:\n")
display(aqi_rules.head(10))


Top Rules Explaining AQI Behavior:



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


In [ ]:

severe_rules = aqi_rules[
    aqi_rules["consequents"].astype(str).str.contains("Severe")
].sort_values("lift", ascending=False)

print("\nTOP RULES LEADING TO SEVERE AQI:\n")
display(severe_rules.head(10))



TOP RULES LEADING TO SEVERE AQI:



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


In [ ]:
good_rules = aqi_rules[
    aqi_rules["consequents"].astype(str).str.contains("Good")
].sort_values("lift", ascending=False)

print("\nTOP RULES LEADING TO GOOD AQI:\n")
display(good_rules.head(10))



TOP RULES LEADING TO GOOD AQI:



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski


In [ ]:
df['Season'] = df['Month'].map({
    12:"Winter",1:"Winter",2:"Winter",
    3:"Summer",4:"Summer",5:"Summer",
    6:"Monsoon",7:"Monsoon",8:"Monsoon",
    9:"Post-Monsoon",10:"Post-Monsoon",11:"Post-Monsoon"
})

seasonal_summary = df.groupby("Season")[
    ["PM2.5","PM10","NO2","SO2","Ozone","CO","AQI"]
].mean()

print("\n🍂 SEASONAL POLLUTION PATTERNS (Interpretation for Apriori Rules):\n")
display(seasonal_summary)



🍂 SEASONAL POLLUTION PATTERNS (Interpretation for Apriori Rules):



,PM2.5,PM10,NO2,SO2,Ozone,CO,AQI
Season,,,,,,,
Monsoon,49.332310,123.672962,29.972636,19.238043,41.175163,0.658505,114.173913
Post-Monsoon,89.086236,229.459835,44.714148,21.406319,30.127170,1.159148,206.854396
Summer,91.909783,228.666060,29.077690,24.261304,46.296386,0.958940,207.236413
Winter,133.565429,292.615512,45.209695,15.439418,27.521524,1.334044,282.149584


In [ ]:
min_severe = severe_rules.copy()
min_severe["antecedent_len"] = min_severe["antecedents"].apply(lambda x: len(x))


min_severe_conditions = min_severe.sort_values(
    ["antecedent_len", "confidence", "lift"],
    ascending=[True, False, False]
)

print("\nMINIMUM CONDITIONS THAT CAUSE SEVERE AQI:\n")
display(min_severe_conditions.head(10))



MINIMUM CONDITIONS THAT CAUSE SEVERE AQI:



,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,representativity,leverage,conviction,zhangs_metric,jaccard,certainty,kulczynski,antecedent_len


In [ ]:
print("Apriori analysis completed successfully.")
print("This notebook expands the previous predictive models by revealing the hidden pollutant combinations that lead to Good, Poor, or Severe AQI levels.")


Apriori analysis completed successfully.
This notebook expands the previous predictive models by revealing the hidden pollutant combinations that lead to Good, Poor, or Severe AQI levels.
